### Load mat data

In [1]:
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from __future__ import division
from sklearn.decomposition import FactorAnalysis as factan

In [2]:
mat_contents = sio.loadmat('/Users/dalin/Documents/My_Research/Dataset/ALM_Svoboda_Lab_Data/Code/TLDS/TempDat/Simultaneous_Spikes.mat')

In [3]:
nDataSet = mat_contents['nDataSet']
params = mat_contents['params']

In [4]:
# nDataSet[0, 0][0].flatten()[0] # sessionIndex
# nDataSet[0, 0][1].flatten() # nUnit
# nDataSet[0, 0][2] # unitROCIndex
# nDataSet[0, 0][3].flatten() # unit_yes_trial_index
# nDataSet[0, 0][4].flatten() # unit_no_trial_index
# nDataSet[0, 0][5] # unit_yes_trial, trial x unit x Time
# nDataSet[0, 0][6] # unit_no_trial, trial x unit x Time
# nDataSet[0, 0][7].flatten() # totTargets
# nDataSet[0, 0][8].flatten() # firstLickTime

In [5]:
nSession = 17 # matlab index
nSession = nSession - 1
totTargets = nDataSet[0, nSession][7].flatten()
unit_yes_trial = nDataSet[0, nSession][5]
unit_no_trial = nDataSet[0, nSession][6]

In [6]:
from pybasicbayes.util.text import progprint_xrange
from pylds.util import random_rotation
from pyslds.models import DefaultSLDS
import numpy.random as npr

In [7]:
unit_trial = np.concatenate((unit_yes_trial, unit_no_trial))
numTrial, numUnit, numTime = unit_trial.shape

In [8]:
factor_unit_trial = unit_trial.transpose([0, 2, 1])
factor_unit_trial = factor_unit_trial.reshape([-1, factor_unit_trial.shape[2]])

In [9]:
# D_init = factor_unit_trial.mean(axis=0)

In [10]:
# factor_unit_trial = factor_unit_trial - D_init

In [11]:
np.random.seed(12345678)
# transition
K = 8
yDim = numUnit
xDim = 5
inputDim = 1 # some constants
inputs = np.ones((numTime, inputDim))

In [12]:
estimator = factan(n_components=xDim, tol=0.000001, copy=True, 
                   max_iter=1000, noise_variance_init=None, 
                   svd_method='randomized', iterated_power=3, 
                   random_state=None)

In [13]:
estimator.fit(factor_unit_trial)
C_init = estimator.components_.T
D_init = estimator.mean_

In [14]:
Cs=[C_init.copy() for _ in range(K)]
Ds=[D_init.copy().reshape([-1, 1]) for _ in range(K)]
# Cs = C_init.copy()
# Ds = D_init.copy().reshape([-1, 1])

In [15]:
test_model = DefaultSLDS(K, yDim, xDim, inputDim,
                         Cs=Cs,
                         Ds=Ds)
for trial in range(numTrial):
    test_model.add_data(unit_trial[trial].T, inputs=inputs)

In [16]:
print("Initializing with Gibbs")
N_gibbs_samples = 1000
def initialize(model):
    model.resample_model()
    return model.log_likelihood()

gibbs_lls = [initialize(test_model) for _ in progprint_xrange(N_gibbs_samples)]

Initializing with Gibbs
.........................  [   25/1000,    1.32sec avg, ETA 21:23 ]
.........................  [   50/1000,    1.49sec avg, ETA 23:33 ]
.........................  [   75/1000,    1.61sec avg, ETA 24:46 ]
.........................  [  100/1000,    1.67sec avg, ETA 24:60 ]
.........................  [  125/1000,    1.66sec avg, ETA 24:14 ]
.........................  [  150/1000,    1.70sec avg, ETA 24:05 ]
.........................  [  175/1000,    1.72sec avg, ETA 23:39 ]
.........................  [  200/1000,    1.71sec avg, ETA 22:46 ]
.........................  [  225/1000,    1.71sec avg, ETA 22:04 ]
.........................  [  250/1000,    1.71sec avg, ETA 21:22 ]
.........................  [  275/1000,    1.72sec avg, ETA 20:44 ]
.........................  [  300/1000,    1.72sec avg, ETA 20:01 ]
.........................  [  325/1000,    1.72sec avg, ETA 19:21 ]
.........................  [  350/1000,    1.72sec avg, ETA 18:36 ]
........................

In [17]:
# Fit with VBEM
print("Fitting with VBEM")
N_vbem_iters = 1000
def update(model):
    model.VBEM_step()
    return model.log_likelihood()

test_model.states_list[0]._init_mf_from_gibbs()
vbem_lls = [update(test_model) for _ in progprint_xrange(N_vbem_iters)]

Fitting with VBEM


AttributeError: 'HMMSLDSStatesEigen' object has no attribute 'E_init_stats'

In [19]:
Cs

array([[ 1.52394402e+00, -1.19598960e+00, -6.98868650e-01,
         7.15491235e-02, -9.84654039e-01],
       [-2.48352773e-01, -2.69510072e-01,  7.17553827e-01,
        -5.81418777e-02,  5.64970032e-01],
       [-2.10174806e-01,  9.46998122e-01,  2.51738865e+00,
         4.73721009e-01,  1.58155755e+00],
       [ 2.85484004e+01, -3.82905302e-01,  2.35230351e-01,
        -7.13216798e-02,  4.33830115e-03],
       [-3.47214476e+00, -8.94549923e+00,  1.22702232e+01,
         6.48830903e+00,  4.43128135e-01],
       [ 3.93464056e+00,  4.81497171e+00, -7.86544390e+00,
         1.03006550e+01, -3.89399509e-02],
       [ 4.49862022e-01,  1.87217259e-01, -2.77787054e-01,
         1.10530903e+00, -4.16636127e-01],
       [ 1.25689610e+00,  3.72946864e-01,  9.30882796e-02,
         1.42541683e+00,  6.38062975e-01],
       [ 1.15181090e+00,  4.65859594e-01, -1.77063441e-02,
         4.17405047e-01, -2.61333751e-01],
       [-1.25578635e+00, -5.62460413e-01,  1.63389342e+00,
         6.95625784e-01

In [18]:
dir(test_model)

['BIC',
 'EM_fit',
 'EM_step',
 'MAP_EM_fit',
 'MAP_EM_step',
 'VBEM_ELBO',
 'VBEM_step',
 'Viterbi_EM_fit',
 'Viterbi_EM_step',
 '_EM_fit',
 '_E_step',
 '_M_step',
 '_M_step_dynamics_distn',
 '_M_step_emission_distn',
 '_M_step_init_dynamics_distn',
 '_M_step_init_state_distn',
 '_M_step_obs_distns',
 '_M_step_trans_distn',
 '_Viterbi_E_step',
 '_Viterbi_M_step',
 '_Viterbi_M_step_init_state_distn',
 '_Viterbi_M_step_obs_distns',
 '_Viterbi_M_step_trans_distn',
 '__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__doc__',
 '__format__',
 '__getattribute__',
 '__getstate__',
 '__hash__',
 '__init__',
 '__long__',
 '__module__',
 '__native__',
 '__new__',
 '__nonzero__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__unicode__',
 '__weakref__',
 '_abc_cache',
 '_abc_negative_cache',
 '_abc_negative_cache_version',
 '_abc_registry',
 '_clear_caches',
 '_emission_distn',
 '_fig_sz',
 '_generate_obs',
 '_ge

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

# Fancy plotting
try:
    import seaborn as sns
    from hips.plotting.colormaps import gradient_cmap
    sns.set_style("white")
    sns.set_context("paper")

    color_names = ["red",
                   "windows blue",
                   "medium green",
                   "dusty purple",
                   "orange",
                   "amber",
                   "clay",
                   "pink",
                   "greyish",
                   "light cyan",
                   "steel blue",
                   "forest green",
                   "pastel purple",
                   "mint",
                   "salmon",
                   "dark brown"]
    colors = sns.xkcd_palette(color_names)
    cmap = gradient_cmap(colors)
except:
    from matplotlib.cm import get_cmap
    colors = ['b', 'r', 'y', 'g', 'purple']
    cmap = get_cmap("jet")

In [ ]:
# Plot the log likelihoods
plt.figure(figsize=(10,6))
# plt.plot(np.arange(N_gibbs_samples), gibbs_lls, color=colors[0], label="gibbs")
# plt.plot(np.arange(N_gibbs_samples + 1, N_gibbs_samples + N_vbem_iters), vbem_lls[1:], color=colors[1], label="vbem")
# plt.xlim(0, N_gibbs_samples + N_vbem_iters)
plt.plot(gibbs_lls, color=colors[0], label="gibbs")
plt.plot(np.array(vbem_lls), color=colors[1], label="vbem")
# plt.xlim([950, 1000])
plt.xlabel('iteration')
plt.ylabel('log likelihood')
plt.legend(loc="lower right")
plt.tight_layout()
# plt.savefig("aux/demo_ll.png")

In [ ]:
# Smooth the data
smoothed_data = test_model.smooth(unit_trial, inputs=inputs)

In [ ]:
fig = plt.figure(figsize=(10,80))
gs = GridSpec(3, 1, height_ratios=[.1, .1, 1.0])
ax = fig.add_subplot(gs[1,0])
timeRange = params['timeSeries'][0][0].flatten()
ax.imshow(test_model.states_list[0].stateseq.reshape(numTrial, numTime), vmin=0, vmax=max(len(colors), test_model.num_states)-1,
          cmap=cmap, interpolation="nearest", extent=[timeRange.min(),timeRange.max(),numTrial,1], aspect='auto')
ax.set_xticks([timeRange.min(), -2.6, -1.2, 0,timeRange.max()])
ax.set_yticks([numTrial, totTargets.sum(),1])
ax.set_xlabel('Time from movement(sec)')
ax.set_ylabel('Trial index')
ax.set_title("Inferred Discrete States")
plt.show()

# ax = fig.add_subplot(gs[2,0])
# plt.plot(y[:,0], color='k', lw=2, label="observed")
# plt.plot(smoothed_data[:,0], color=colors[0], lw=1, label="smoothed")
# plt.xlabel("Time")
# plt.xlim(0, min(T, 500))
# plt.ylabel("Observations")
# plt.legend(loc="upper center", ncol=2)
# plt.tight_layout()
# # d
# # 
# plt.figure()
# from pyhsmm.util.general import rle
# z_rle = rle(z)
# offset = 0
# for k, dur in zip(*z_rle):
#     plt.plot(x[offset:offset+dur,0], x[offset:offset+dur,1], color=colors[k])
#     offset += dur

# plt.xlabel("$x_1$")
# plt.ylabel("$x_2$")
# plt.title("Continuous Latent States")
# plt.show()